In [9]:
import os, re, json, time

import datetime as dt
from dateutil import relativedelta

import tweepy

folder_main = os.getcwd()
folder_creds = os.path.join(folder_main, "creds", "twitter")
folder_dump_main = os.path.join(folder_main, "dump")

query = ["new year resolution", "2020 is the year", "#resolutions2020"]

foldername = "_".join([q.replace(" ", "") for q in query])
folder_dump_query = os.path.join(folder_dump_main, foldername)

if foldername not in os.listdir(folder_dump_main):
    os.mkdir(folder_dump_query)

for file in os.listdir(folder_creds):
    filepath = os.path.join(folder_creds, file)
    with open(filepath, "r") as f:
        globals()[file.split(".")[0]] = json.load(f)

auth = tweepy.OAuthHandler(consumer_key=consumer["key"], consumer_secret=consumer["secret"])
auth.set_access_token(key=token["key"], secret=token["secret"])

api = tweepy.API(auth)

class StreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        print(status.text)
    
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
    def on_data(self, data):
        raw_tweet = json.loads(data)
        
        try:
            with open(os.path.join(folder_dump_query, f"{raw_tweet['id']}.json"), "w") as f:
                json.dump(raw_tweet, f)
        except:
            pass
        
    def start_stream(stream, **kwargs):
        try:
            stream.filter(**kwargs)
        except ReadTimeoutError:
            stream.disconnect()
            logger.exception("ReadTimeoutError exception")
            start_stream(stream, **kwargs)

listener = StreamListener()

stream = tweepy.Stream(auth=api.auth, 
                       listener=listener)

stream.filter(track=query, 
              languages=["en"], 
              is_async=True)